In [ ]:
!pip install -qU rich
!pip install -qU wandb
!pip install -qU git+https://github.com/wandb/weave.git@feat/groq
!pip install -qU llama-index groq
!pip install -qU llama-index-embeddings-huggingface

In [ ]:
import os
from typing import Optional, Tuple

import rich
import wandb
import weave
from google.colab import userdata

from groq import Groq
from llama_index.core import (
    ServiceContext, StorageContext, load_index_from_storage
)
from llama_index.core.base.base_retriever import BaseRetriever
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [ ]:
weave.init(project_name="groq-rag")

artifact = wandb.Api().artifact(
    "geekyrakshit/groq-rag/ncert-flamingoes-prose-embeddings:latest"
)
artifact_dir = artifact.download()

In [ ]:
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

service_context = ServiceContext.from_defaults(
    embed_model=embed_model, llm=None
)

In [ ]:
storage_context = StorageContext.from_defaults(persist_dir=artifact_dir)
index = load_index_from_storage(
    storage_context, service_context=service_context
)

In [ ]:
retreival_engine = index.as_retriever(
    service_context=service_context,
    similarity_top_k=10,
)

In [ ]:
query = """what was the mood in the classroom when M. Hamel gave his last French lesson?"""
response = retreival_engine.retrieve(query)

In [ ]:
chapter_name = response[0].node.metadata["file_name"].split(".")[0].replace("_", " ").title()
context = response[0].node.text

rich.print(f"{chapter_name=}")
rich.print(f"{context=}")

In [ ]:
class EnglishDoubtClearningAssistant(weave.Model):
    model: str = "llama3-8b-8192"
    _groq_client: Optional[Groq] = None
    
    def __init__(self, model: Optional[str] = None):
        super().__init__()
        self.model = model if model is not None else self.model
        self._groq_client = Groq(
            api_key=os.environ.get("GROQ_API_KEY")
        )
    
    @weave.op()
    def predict(self, question: str, context: str) -> Tuple[str, str]:
        chat_completion = self._groq_client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": """
You are a student in a class and your teacher has asked you to answer the following question.
You have to write the answer in the given word limit.""",
                },
                {
                    "role": "user",
                    "content": f"""
We have provided context information below. 

---
{context}
---

Answer the following question within 50-150 words:

```
{query}
```""",
                },
            ],
            model=self.model,
        )
        return chat_completion.choices[0].message.content

In [ ]:
assistant = EnglishDoubtClearningAssistant()

rich.print(assistant.predict(question=query, context=context))

In [ ]:
class EnglishTeachingAssistant(weave.Model):
    embeddings_artifact_wandb_address: Optional[str] = None
    embedding_model_huggingface_address: Optional[str] = None
    _groq_client: Optional[Groq] = None
    _retreival_engine: Optional[BaseRetriever] = None

    def __init__(
        self,
        embeddings_artifact_wandb_address: Optional[
            str
        ] = "geekyrakshit/groq-rag/ncert-flamingoes-prose-embeddings:latest",
        embedding_model_huggingface_address: Optional[str] = "BAAI/bge-small-en-v1.5",
    ):
        super().__init__()
        self.embeddings_artifact_wandb_address = embeddings_artifact_wandb_address
        self.embedding_model_huggingface_address = embedding_model_huggingface_address
        self._build_vector_store()
        self._groq_client = Groq(
            api_key=os.environ.get("GROQ_API_KEY"),
        )

    def _build_vector_store(self):
        artifact = wandb.Api().artifact(
            "geekyrakshit/groq-rag/ncert-flamingoes-prose-embeddings:latest"
        )
        artifact_dir = artifact.download()
        service_context = ServiceContext.from_defaults(
            embed_model=HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5"),
            llm=None,
        )
        storage_context = StorageContext.from_defaults(persist_dir=artifact_dir)
        index = load_index_from_storage(
            storage_context, service_context=service_context
        )
        self._retreival_engine = index.as_retriever(
            service_context=service_context,
            similarity_top_k=10,
        )

    @weave.op()
    def get_response_from_groq(self, system_prompt: str, user_prompt: str) -> str:
        chat_completion = self._groq_client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": system_prompt,
                },
                {
                    "role": "user",
                    "content": user_prompt,
                },
            ],
            model="llama3-8b-8192",
        )
        return chat_completion.choices[0].message.content

    @weave.op()
    def get_prompt_for_emulating_student_response(
        self, question: str, context: str, word_limit_min: int, word_limit_max: int
    ) -> Tuple[str, str]:
        user_prompt = f"""
        We have provided context information below. 

        ---
        {context}
        ---

        Answer the following question within {word_limit_min}-{word_limit_max} words:

        ```
        {question}
        ```
        """
        system_prompt = """
        You are a student in a class and your teacher has asked you to answer the following question.
        You have to write the answer in the given word limit.
        """
        return user_prompt, system_prompt

    @weave.op()
    def get_prompt_for_doubt_clearning(
        self, question: str, context: str
    ) -> Tuple[str, str]:
        user_prompt = f"""
        We have provided context information below. 

        ---
        {context}
        ---

        Answer the following question as helpfully as you can:

        ```
        {question}
        ```
        """
        system_prompt = """
        You are a helpful assistant to an English teacher meant to solve the doubts of students.
        You have to write the answer to the following question as helpfully as you can.
        """
        return user_prompt, system_prompt

    @weave.op()
    def format_doubt_clearning_response(
        self, answer: str, context: str, chapter_name: str
    ) -> str:
        return f"""
        The answer is:
        
        {answer}
        
        The answer is based on the following excerpt from the chapter {chapter_name} from the NCERT Flamingo textbook:

        {context}
        """

    @weave.op()
    def predict(
        self,
        question: str,
        total_marks: Optional[int] = None,
        grade_response: Optional[bool] = False,
    ) -> str:
        response = self._retreival_engine.retrieve(question)
        chapter_name = (
            response[0]
            .node.metadata["file_name"]
            .split(".")[0]
            .replace("_", " ")
            .title()
        )
        context = response[0].node.text
        if total_marks:
            if total_marks < 3:
                word_limit_min = 5
                word_limit_max = 50
            elif total_marks < 5:
                word_limit_min = 50
                word_limit_max = 100
            else:
                word_limit_min = 100
                word_limit_max = 200
        else:
            word_limit_min = None
            word_limit_max = None

        if total_marks:
            if not grade_response:
                (
                    user_prompt,
                    system_prompt,
                ) = self.get_prompt_for_emulating_student_response(
                    question, context, word_limit_min, word_limit_max
                )
                return self.get_response_from_groq(
                    system_prompt=system_prompt, user_prompt=user_prompt
                )

        user_prompt, system_prompt = self.get_prompt_for_doubt_clearning(
            question, context
        )
        answer = self.get_response_from_groq(
            system_prompt=system_prompt, user_prompt=user_prompt
        )
        return self.format_doubt_clearning_response(answer, context, chapter_name)

In [ ]:
assistant = EnglishTeachingAssistant()

In [ ]:
rich.print(assistant.predict(question=query))

In [ ]:
rich.print(assistant.predict(question=query, marks=5))